In [304]:
f = open("data.txt", "r")
datastr = f.read()
datalines = datastr.split("\n")


In [305]:
height_grid = []
for y in datalines:
    height_grid.append(list(y))

def create_grid(w, h, val):
    eg = []
    for j in range(h):
        eg.append([val] * w)
    return eg

H = len(datalines)
W = len(datalines[0])
INFINITY = 9999

score_grid = create_grid(W,H,INFINITY)

dir_dict = {0: [0, -1], 1: [1, 0], 2: [0, 1], 3: [-1, 0]}

In [306]:
def locate_start_end_points():
    sx, sy, ex, ey = -1, -1, -1, -1
    for i, l in enumerate(datalines):
        x = l.find("S")
        if x != -1:
            sy = i
            sx = x

    for i, l in enumerate(datalines):
        x = l.find("E")
        if x != -1:
            ey = i
            ex = x
    return sx, sy, ex, ey
(START_X, START_Y, END_X, END_Y) = locate_start_end_points()

In [307]:
def step_value(letter_on, letter_side):
    HEIGHT_ORDER = "SabcdefghijklmnopqrstuvwxyzE" # S for start, E for end
    if len(letter_on) != 1 or len(letter_side) != 1:
        print("Error legal step")
        return False
    i_on = HEIGHT_ORDER.find(letter_on)
    i_side = HEIGHT_ORDER.find(letter_side)

    return i_side - i_on

In [308]:
def probe_neighbours(grid, cx, cy):
    if cy-1 >= 0:
        north = grid[cy-1][cx]
    else:
        north = None
    if cy+1 < len(grid):
        south = grid[cy+1][cx]
    else:
        south = None
    if cx-1 >= 0:
        west = grid[cy][cx-1]
    else:
        west = None
    if cx+1 < len(grid[0]):
        east = grid[cy][cx+1]
    else:
        east = None
    return (north, east, south, west)

#### Dijkstra's algorithm for maze solving (Wikipedia)
Let the node at which we are starting be called the initial node. Let the distance of node Y be the distance from the initial node to Y. Dijkstra's algorithm will initially start with infinite distances and will try to improve them step by step.

1) Mark all nodes unvisited. Create a set of all the unvisited nodes called the unvisited set.
2) Assign to every node a tentative distance value: set it to zero for our initial node and to infinity for all other nodes. During the run of the algorithm, the tentative distance of a node v is the length of the shortest path discovered so far between the node v and the starting node. Since initially no path is known to any other vertex than the source itself (which is a path of length zero), all other tentative distances are initially set to infinity. Set the initial node as current.[16]
3) For the current node, consider all of its unvisited neighbors and calculate their tentative distances through the current node. Compare the newly calculated tentative distance to the one currently assigned to the neighbor and assign it the smaller one. For example, if the current node A is marked with a distance of 6, and the edge connecting it with a neighbor B has length 2, then the distance to B through A will be 6 + 2 = 8. If B was previously marked with a distance greater than 8 then change it to 8. Otherwise, the current value will be kept.
4) When we are done considering all of the unvisited neighbors of the current node, mark the current node as visited and remove it from the unvisited set. A visited node will never be checked again (this is valid and optimal in connection with the behavior in step 6.: that the next nodes to visit will always be in the order of 'smallest distance from initial node first' so any visits after would have a greater distance).
5) If the destination node has been marked visited (when planning a route between two specific nodes) or if the smallest tentative distance among the nodes in the unvisited set is infinity (when planning a complete traversal; occurs when there is no connection between the initial node and remaining unvisited nodes), then stop. The algorithm has finished.
6) Otherwise, select the unvisited node that is marked with the smallest tentative distance, set it as the new current node, and go back to step 3.

In [309]:
# Step 2
nx = START_X
ny = START_Y
score_grid[ny][nx] = 0
visited = create_grid(W,H,-1)

counter = 0
while(counter < H*W):
    counter += 1
    # Step 3

    # legal nodes: return values: (North, East, South, West)
    neighbour_visited = probe_neighbours(visited, nx, ny)
    neighbour_scores = probe_neighbours(score_grid, nx, ny)

    # fill score values around current node
    for i in range(4):
        if neighbour_scores[i] != None:
            v = dir_dict[i]
            if step_value(height_grid[ny][nx], height_grid[ny+v[1]][nx+v[0]]) <= 1: # step is legal
                new_val = score_grid[ny][nx] + 1
                old_val = score_grid[ny+v[1]][nx+v[0]]
                if new_val < old_val:
                    score_grid[ny+v[1]][nx+v[0]] = new_val


    # Step 4)
    visited[ny][nx] = 1

    # Step 5) 
    neighbour_visited = probe_neighbours(visited, nx, ny)
    neighbour_scores = probe_neighbours(score_grid, nx, ny)

    found_new_cell = False
    for i in range(4):
        if neighbour_visited[i] != None and neighbour_visited[i] != 1 and neighbour_scores[i] < INFINITY: 
            v = dir_dict[i]
            nx += v[0]
            ny += v[1]
            found_new_cell = True   
            break
    
    if not found_new_cell:
        for i in range(W):
            for j in range(H):
                if score_grid[j][i] < INFINITY and visited[j][i] == -1:
                    nx = i
                    ny = j

    # if not found_new_cell:
    #     # max(max(score_grid)) == INFINITY:
    #     minimum = min(min(score_grid))
    #     while (minimum < INFINITY*0.1):
    #         for i in range(W):
    #             for j in range(H):
    #                 neighbour_scores = probe_neighbours(score_grid, i, j)
                    
    #                 if score_grid[j][i] == minimum and neighbour_scores.__contains__(INFINITY):
    #                     nx = i
    #                     ny = j
    #                     break
    #         minimum += 1
    
    # print("--"*50)
    # for l in score_grid:
    #     print(l[0:10])
    # print("--"*50)
    


In [312]:
for l in score_grid:
    print(l)

print("Shortest distance:", score_grid[END_Y][END_X])

[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 9999, 9999, 34, 35, 52, 51, 50, 51, 52, 53, 54, 55, 56, 61, 62, 63, 64, 65, 66, 67, 68, 69, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 151, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 184, 183, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260]
[19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 52, 51, 50, 49, 50, 51, 54, 55, 56, 61, 60, 61, 62, 65, 66, 67, 68, 69, 70, 77, 78, 79, 80, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 1

In [ ]:
# 590 is too high